In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

Read input data

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

Fill unknown values on mean value

In [4]:
train_df["Embarked"] = train_df["Embarked"].fillna("S")
train_df["Age"].fillna(train_df["Age"].mean(), inplace=True)
train_df["Fare"].fillna(train_df["Fare"].mean(), inplace=True)

Drop too personal column

In [5]:
X_train = train_df.drop("PassengerId", axis=1).drop("Survived", axis=1).drop("Name", axis=1).drop("Ticket", axis=1).drop("Cabin", axis=1).copy()
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).drop("Name", axis=1).drop("Ticket", axis=1).drop("Cabin", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape
print "ok"

ok


In [6]:
X_train.tail()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
886,2,male,27.000000,0,0,13.00,S
887,1,female,19.000000,0,0,30.00,S
888,3,female,29.699118,1,2,23.45,S
889,1,male,26.000000,0,0,30.00,C
890,3,male,32.000000,0,0,7.75,Q


CatBoost can work with categorial features, we don't need to bring "Sex" and "Embarked" to integer values.

In [8]:
model = CatBoostRegressor(learning_rate=1, depth=12, loss_function='RMSE')
cat_features = [1, 6]
fit_model = model.fit(X_train, Y_train, cat_features)

print fit_model.get_params()

Y_test = fit_model.predict(X_test)

{'rsm': 1, 'use_best_model': False, 'random_seed': 268362308039490, 'name': u'experiment', 'eval_metric': u'RMSE', 'thread_count': 4, 'feature_border_type': u'MinEntropy', 'learning_rate': 1, 'random_strength': 1, 'verbose': False, 'depth': 12, 'auto_stop_pval': 0, 'iterations': 500, 'l2_leaf_reg': 3, 'leaf_estimation_method': u'Gradient', 'gradient_iterations': 1, 'store_all_simple_ctr': False, 'has_time': False, 'loss_function': u'RMSE'}


In [9]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_test.astype(np.int64)
    })
submission.to_csv("submission.csv", index=False)